In [1]:
! pip install google-generativeai pydantic


  Using cached pydantic-2.11.3-py3-none-any.whl.metadata (65 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached typing_inspection-0.4.0-py3-none-any.whl.metadata (2.6 kB)
  Using cached requests-2.32.3-py3-none-any.whl.metadata (4.6 kB)
  Using cached rsa-4.9-py3-none-any.whl.metadata (4.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
  Using cached urllib3-2.4.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached certifi-2025.1.31-py3-none-any.whl.metadata (2.5 kB)
INFO: pip is looking at multiple versions of grpcio-status to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/1.3 MB ? eta -:--:--
   ---------------------------------------- 1.3/1.3 MB 33.3 MB/s eta 0:00:00
Using cached pydantic-2.11.3-py3-none-any.whl (443 k

In [11]:
import google.generativeai as genai
import json
from dotenv import load_dotenv
import os

load_dotenv()
API_KEY = os.getenv("GEMINI_API_KEY")  # Changed to match common .env variable name

if not API_KEY:
    raise ValueError("GEMINI_API_KEY not found in .env file")

# Configure the Gemini client (only needed once in your app)
genai.configure(api_key=API_KEY)  # Replace with your actual API key

# Use the 1.5 Flash model with JSON output
model = genai.GenerativeModel(
    "gemini-1.5-flash",
    generation_config={"response_mime_type": "application/json"}
)

# Prompt Gemini to follow a simple schema
prompt = """
List 5 popular cookie recipes.
Use this JSON schema:
  Recipe = {"recipe_name": str , "ingredients": list[str]}
Return a `list[Recipe]`.
"""

# Generate and parse the response
response = model.generate_content(prompt)

# Safely parse the JSON from the response text
try:
    data = json.loads(response.text)
    print(json.dumps(data, indent=2))
except Exception as e:
    print("Failed to parse JSON:", e)
    print("Raw response:\n", response.text)

[
  {
    "recipe_name": "Chocolate Chip Cookies",
    "ingredients": [
      "butter",
      "sugar",
      "eggs",
      "flour",
      "chocolate chips"
    ]
  },
  {
    "recipe_name": "Oatmeal Raisin Cookies",
    "ingredients": [
      "butter",
      "sugar",
      "eggs",
      "flour",
      "oats",
      "raisins"
    ]
  },
  {
    "recipe_name": "Peanut Butter Cookies",
    "ingredients": [
      "peanut butter",
      "sugar",
      "eggs",
      "flour"
    ]
  },
  {
    "recipe_name": "Snickerdoodles",
    "ingredients": [
      "butter",
      "sugar",
      "eggs",
      "flour",
      "cinnamon",
      "cream of tartar"
    ]
  },
  {
    "recipe_name": "Gingerbread Cookies",
    "ingredients": [
      "butter",
      "sugar",
      "eggs",
      "flour",
      "ginger",
      "molasses"
    ]
  }
]


In [13]:
data=data

In [15]:
from pydantic import BaseModel
from typing import List
import json

# Define Pydantic models
class Recipe(BaseModel):
    ingredients: List[str]
    recipe_name: str

class RecipeList(BaseModel):
    recipes: List[Recipe]

# Create Pydantic model instance
recipe_list = RecipeList(recipes=[Recipe(**recipe) for recipe in data])

# Access and print recipe data
for recipe in recipe_list.recipes:
    print(f"Recipe: {recipe.recipe_name}")
    print(f"Ingredients: {recipe.ingredients}")
    print()

# Accessing a specific recipe
print("First recipe:")
print(recipe_list.recipes[0].recipe_name)
print(recipe_list.recipes[0].ingredients)

Recipe: Chocolate Chip Cookies
Ingredients: ['butter', 'sugar', 'eggs', 'flour', 'chocolate chips']

Recipe: Oatmeal Raisin Cookies
Ingredients: ['butter', 'sugar', 'eggs', 'flour', 'oats', 'raisins']

Recipe: Peanut Butter Cookies
Ingredients: ['peanut butter', 'sugar', 'eggs', 'flour']

Recipe: Snickerdoodles
Ingredients: ['butter', 'sugar', 'eggs', 'flour', 'cinnamon', 'cream of tartar']

Recipe: Gingerbread Cookies
Ingredients: ['butter', 'sugar', 'eggs', 'flour', 'ginger', 'molasses']

First recipe:
Chocolate Chip Cookies
['butter', 'sugar', 'eggs', 'flour', 'chocolate chips']
